In [1]:
import streamlit as st
import pandas as pd

from googleapiclient.discovery import build
from google.oauth2 import service_account
from io import BytesIO
from googleapiclient.http import MediaIoBaseDownload

from datetime import datetime
import pytz

import plotly.express as px

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = '../service_account.json'
PARENT_FOLDER_ID = '1vIKk9xgUn1JGY2MFPdB6W3tEfU8oVptP'

def authenticate():
    '''
    Autentikasi akun untuk akses ke gdrive
    '''
    creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return creds

def list_files():
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    results = service.files().list(
        q=f"'{PARENT_FOLDER_ID}' in parents",
        spaces='drive',
        fields='nextPageToken, files(id, name, modifiedTime)',
        pageSize=10
    ).execute()
    
    items = results.get('files', [])
    
    file_list = []
    if items:
        file_list = [(item['name'], item['id'], item['modifiedTime']) for item in items]
    return file_list

def read_file_from_drive(file_id):
    '''
    Read Excel file from Google Drive
    '''
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    # Stream the file content
    request = service.files().get_media(fileId=file_id)
    file_data = BytesIO()
    downloader = MediaIoBaseDownload(file_data, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    
    file_data.seek(0)
    return file_data  # Return the raw file data

def read_file_from_drive(file_id):
    '''
    Read Excel file from Google Drive
    '''
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    # Stream the file content
    request = service.files().get_media(fileId=file_id)
    file_data = BytesIO()
    downloader = MediaIoBaseDownload(file_data, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    
    file_data.seek(0)
    return file_data  # Return the raw file data

def processing_excel(file_data, sheet_name):
    '''
    Process Excel file to clean and prepare the data
    '''
    
    excel_ptr = pd.read_excel(file_data, sheet_name, header=None)
    temp_df = excel_ptr.ffill()
    
    listof_ver = temp_df[temp_df[1].str.contains('PTR Ver', na=False)][1].unique().tolist()
    
    # Get the rows which will become header
    value_to_skip = 'Features'
    max_rows_to_scan = 10

    header_index = excel_ptr.head(max_rows_to_scan).apply(lambda row: row.astype(str).str.contains(value_to_skip).any(), axis=1).idxmax()

    if pd.isna(header_index):  # If 'Features' wasn't found in the scanned rows
        print("Header 'Features' not found in the first", max_rows_to_scan, "rows.")
    else:
        new_header = excel_ptr.iloc[header_index].values
        excel_ptr = excel_ptr.iloc[header_index+1:].copy()
        excel_ptr.columns = new_header
        
    excel_ptr = excel_ptr.iloc[:, 1:]
    excel_ptr.reset_index(drop=True, inplace=True)
    
    # Convert column OS Version types
    if 'OS Version' not in excel_ptr.columns:
        st.warning("The selected sheet does not have an 'OS Version' column. Skipping this part of processing.")
    else:
        excel_ptr['OS Version'] = excel_ptr['OS Version'].astype(str)
    
    # drop column Number
    excel_ptr.drop(columns='No', inplace=True, errors='ignore')
    
    #Rename the column
    excel_ptr.rename(columns={
        'Sub Fitur': 'Sub-features',
        'Rekening Sumber\n[Jika ada]': 'Rekening Sumber',
        'Data yang Digunakan\n[Jika ada]': 'Data yang digunakan',
        'FT\n[Jika Ada]': 'FT',
        'Tanggal Eksekusi\n[harus diisi]': 'Tanggal Eksekusi',
        'Tanggal Passed\n[harus diisi]': 'Tanggal Passed'
    }, inplace=True)
    
    # Define the columns to ffill
    columns_to_ffill = ['Features', 'Sub-features', 'Expected Condition']

    # Check if 'Link JIRA' exists in the dataframe
    if 'Link JIRA' in excel_ptr.columns:
        columns_to_ffill.append('Link JIRA')

    # Apply ffill only on the selected columns
    excel_ptr[columns_to_ffill] = excel_ptr[columns_to_ffill].apply(lambda x: x.ffill())

    
    return excel_ptr, listof_ver

def status_progress(processed_excel, version):
    """
    Calculate the progress based on the processed Excel data.
    """
    statusBased_OS = [(x, y) for x, y in zip(processed_excel['OS'].values, processed_excel['Status '+ version].values)]
    passed_prog = sum(1 for item in statusBased_OS if item == ('Android', 'Passed'))
    failed_prog = sum(1 for item in statusBased_OS if item == ('Android', 'Failed'))
    unknown_prog = sum(1 for item in statusBased_OS if item == ('Android', 'N/A'))
    inprog_prog = sum(1 for item in statusBased_OS if item == ('Android', 'In Progress'))
    
    return passed_prog, failed_prog, unknown_prog, inprog_prog


In [2]:
file_list = list_files()

In [3]:
file_list

[('PTR Sprint 12 - Fitri (1).xlsx',
  '1Lss1vt8f7yWcvWOTpRebY7Vd1JoQ_A4O',
  '2025-01-07T04:41:36.930Z'),
 ('PTR_Rilis_D.xlsx',
  '1JzRCBOTHgisl-ckN58w9TA_t4bUgMusv',
  '2025-01-05T18:34:10.572Z'),
 ('Testing PTR Rilis D_Hotfix_Prod Issue_(November2024).xlsx',
  '1QIR-Of8MaZT1kd2ZxF1i3PR6b3dFkRU6',
  '2024-12-24T04:54:23.446Z')]

In [4]:
[(file_id, modified_time) for name, file_id, modified_time in file_list if name == 'PTR Sprint 12 - Fitri (1).xlsx'][0][0]

'1Lss1vt8f7yWcvWOTpRebY7Vd1JoQ_A4O'

In [5]:
selected_file_data = [(file_id, modified_time) for name, file_id, modified_time in file_list if name == 'PTR Sprint 12 - Fitri (1).xlsx'][0][0]
selected_file_id = selected_file_data

file_data = read_file_from_drive(selected_file_id)

# Ngetest Manual buat liat mana yang error

In [6]:
file_data

In [7]:
excel_file = pd.ExcelFile(file_data)
excel_file.sheet_names  # Returns a list of sheet names

['Regresi App 6 Jan (1.0.3 Prod)',
 'PTR Impactd Fitur-PO(1.0.3 PTR)',
 'Test Apps 7 Jan-PO (1.0.3 PTR )',
 'PTR fitur baru-PO (1.0.5 PTR)',
 'PTR existing ftr-PO (1.0.5 PTR)',
 'PTR Hotfix-PO (1.0.5 PTR)',
 'Testing PTR-Tester (1.0.5 PTR)',
 'PTR_PO_Payment',
 'Tiket Hotfix PO',
 'PTR PO-Surrounding deploy',
 'Sheet1',
 'PTR PO Payment - Surrounding dp',
 'Sheet2',
 'Testing PTR Backoffice-Tester',
 'Tiket Ready PTR-Tester (1.0.5)',
 '-']

In [137]:
excel_ptr = pd.read_excel(file_data, 'PTR Hotfix-PO (1.0.5 PTR)', header=None)
temp_df = excel_ptr.ffill()

listof_ver = temp_df[temp_df[1].str.contains('PTR Ver', na=False)][1].unique().tolist()

# Get the rows which will become header
value_to_skip = 'Features'
max_rows_to_scan = 20

header_index = excel_ptr.head(max_rows_to_scan).apply(lambda row: row.astype(str).str.contains(value_to_skip).any(), axis=1).idxmax()

if pd.isna(header_index):  # If 'Features' wasn't found in the scanned rows
    print("Header 'Features' not found in the first", max_rows_to_scan, "rows.")
else:
    new_header = excel_ptr.iloc[header_index].values
    excel_ptr = excel_ptr.iloc[header_index+1:].copy()
    excel_ptr.columns = new_header
    
excel_ptr = excel_ptr.iloc[:, 1:]
excel_ptr.reset_index(drop=True, inplace=True)

In [138]:
listof_ver

['PTR Ver APK 1.0.5 IPA 1.0.5']

In [139]:
if 'OS Version' not in excel_ptr.columns:
    st.warning("The selected sheet does not have an 'OS Version' column. Skipping this part of processing.")
else:
    excel_ptr['OS Version'] = excel_ptr['OS Version'].astype(str)

In [140]:
excel_ptr.drop(columns='No', inplace=True, errors='ignore')

In [141]:
#Rename the column
excel_ptr.rename(columns={
    'Sub Fitur': 'Sub-features',
    'Rekening Sumber\n[Jika ada]': 'Rekening Sumber',
    'Data yang Digunakan\n[Jika ada]': 'Data yang digunakan',
    'FT\n[Jika Ada]': 'FT',
    'Tanggal Eksekusi\n[harus diisi]': 'Tanggal Eksekusi',
    'Tanggal Passed\n[harus diisi]': 'Tanggal Passed'
}, inplace=True)

# Because of merged and centered, need to use ffill to duplicate values before current rows
excel_ptr[['Features', 'Sub-features', 'Expected Condition']] = excel_ptr[['Features', 'Sub-features', 'Expected Condition']].apply(lambda x: x.ffill())


In [142]:
excel_ptr

,Link JIRA,Features,Sub-features,Problem Description,Expected Condition,OS,OS Version,Tipe Device HP,Telko Provider HP,Rekening Sumber,Data yang digunakan,PO,FT,Link Report Test,Description Issue dan Evidence,Status PTR PTR Ver APK 1.0.5 IPA 1.0.5,Tanggal Eksekusi PTR Ver APK 1.0.5 IPA 1.0.5,Tanggal Passed PTR Ver APK 1.0.5 IPA 1.0.5,Komentar
0,[PIB-42] Donwload dan Share Resi Transaksi,Share Resi,Share Resi,Nama Nasabah : xxxxxxxxxxxxxxxxANI\nNomor Reke...,Nasabah bisa download dan share resi transaksi,Android,nan,NaN,NaN,NaN,NaN,Ori,NaN,NaN,NaN,Pass with noted,NaN,NaN,NaN
1,NaN,Share Resi,Share Resi,NaN,Nasabah bisa download dan share resi transaksi,iOS,nan,NaN,NaN,NaN,NaN,Ori,NaN,NaN,NaN,Pass with noted,NaN,NaN,NaN
2,[PIB-106] Gagal Permintaan Kartu Debit,Manajemen Kartu,Manajemen Kartu,"Mohon Bantuannya, terdapat kegagalan request k...",Nasabah bisa request kartu debit di iOS dan An...,Android,nan,NaN,NaN,NaN,NaN,"Rifatun, Indah",NaN,NaN,NaN,Passed,NaN,NaN,NaN
3,NaN,Manajemen Kartu,Manajemen Kartu,NaN,Nasabah bisa request kartu debit di iOS dan An...,iOS,nan,NaN,NaN,NaN,NaN,"Rifatun, Indah",NaN,NaN,NaN,Passed,NaN,NaN,NaN
4,[PIB-79] Kartu Debit Baru di Manajemen Kartu,Manajemen Kartu,Manajemen Kartu,Ketika membuat kartu debit baru. Carrousel kar...,Harusnya ada constraint atau pembatas,iOS,nan,NaN,NaN,NaN,NaN,"Rifatun, Indah",NaN,NaN,NaN,Passed,NaN,NaN,NaN
5,[PIB-74] Quick Menu Tabungan Haji di Homescreen,Homescreen,Homescreen,iOS utk tabungan haji fitur dibawah tidak beru...,Harusnya berubah sesuai ngambar SS di Figma ya...,iOS,nan,NaN,NaN,NaN,NaN,"Futsia, arib",NaN,NaN,NaN,Passed,NaN,NaN,NaN
6,[PIB-67] Snackbar E-Money Salah Posisi,E-Money,E-Money,Sedangkan android defect jdinya di belakang ov...,Snackbar harusnya berada di depan overlay hita...,Android,nan,NaN,NaN,NaN,NaN,"Nando, anggoran",NaN,NaN,NaN,Passed,NaN,NaN,NaN
7,[PIB-58] No Rekening Tabungan Emas dan SBSN,"Portofolio, Portofolio Emas, Portofolio SBSN","Portofolio, Portofolio Emas, Portofolio SBSN",di iOS tidak bisa klik icon copy no rekening t...,Seharusnya ada klik icon copy nomer rekening t...,iOS,nan,NaN,NaN,NaN,NaN,Ajeng,NaN,NaN,NaN,Passed,NaN,NaN,NaN
8,[PIB-41] Force Close,Mega Menu,Mega Menu,pada saat melakukan paste di field 'search' te...,Seharusnya tidak force closed,Android,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Passed,NaN,NaN,NaN
9,NaN,Mega Menu,Mega Menu,NaN,Seharusnya tidak force closed,iOS,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Failed,NaN,NaN,NaN


# Ini pake function (ga manual)

In [145]:
df, version = processing_excel(file_data, 'PTR Hotfix-PO (1.0.5 PTR)')
df.head(10)

,Link JIRA,Features,Sub-features,Problem Description,Expected Condition,OS,OS Version,Tipe Device HP,Telko Provider HP,Rekening Sumber,Data yang digunakan,PO,FT,Link Report Test,Description Issue dan Evidence,Status PTR PTR Ver APK 1.0.5 IPA 1.0.5,Tanggal Eksekusi PTR Ver APK 1.0.5 IPA 1.0.5,Tanggal Passed PTR Ver APK 1.0.5 IPA 1.0.5,Komentar
0,[PIB-42] Donwload dan Share Resi Transaksi,Share Resi,Share Resi,Nama Nasabah : xxxxxxxxxxxxxxxxANI\nNomor Reke...,Nasabah bisa download dan share resi transaksi,Android,nan,NaN,NaN,NaN,NaN,Ori,NaN,NaN,NaN,Pass with noted,NaN,NaN,NaN
1,[PIB-42] Donwload dan Share Resi Transaksi,Share Resi,Share Resi,NaN,Nasabah bisa download dan share resi transaksi,iOS,nan,NaN,NaN,NaN,NaN,Ori,NaN,NaN,NaN,Pass with noted,NaN,NaN,NaN
2,[PIB-106] Gagal Permintaan Kartu Debit,Manajemen Kartu,Manajemen Kartu,"Mohon Bantuannya, terdapat kegagalan request k...",Nasabah bisa request kartu debit di iOS dan An...,Android,nan,NaN,NaN,NaN,NaN,"Rifatun, Indah",NaN,NaN,NaN,Passed,NaN,NaN,NaN
3,[PIB-106] Gagal Permintaan Kartu Debit,Manajemen Kartu,Manajemen Kartu,NaN,Nasabah bisa request kartu debit di iOS dan An...,iOS,nan,NaN,NaN,NaN,NaN,"Rifatun, Indah",NaN,NaN,NaN,Passed,NaN,NaN,NaN
4,[PIB-79] Kartu Debit Baru di Manajemen Kartu,Manajemen Kartu,Manajemen Kartu,Ketika membuat kartu debit baru. Carrousel kar...,Harusnya ada constraint atau pembatas,iOS,nan,NaN,NaN,NaN,NaN,"Rifatun, Indah",NaN,NaN,NaN,Passed,NaN,NaN,NaN
5,[PIB-74] Quick Menu Tabungan Haji di Homescreen,Homescreen,Homescreen,iOS utk tabungan haji fitur dibawah tidak beru...,Harusnya berubah sesuai ngambar SS di Figma ya...,iOS,nan,NaN,NaN,NaN,NaN,"Futsia, arib",NaN,NaN,NaN,Passed,NaN,NaN,NaN
6,[PIB-67] Snackbar E-Money Salah Posisi,E-Money,E-Money,Sedangkan android defect jdinya di belakang ov...,Snackbar harusnya berada di depan overlay hita...,Android,nan,NaN,NaN,NaN,NaN,"Nando, anggoran",NaN,NaN,NaN,Passed,NaN,NaN,NaN
7,[PIB-58] No Rekening Tabungan Emas dan SBSN,"Portofolio, Portofolio Emas, Portofolio SBSN","Portofolio, Portofolio Emas, Portofolio SBSN",di iOS tidak bisa klik icon copy no rekening t...,Seharusnya ada klik icon copy nomer rekening t...,iOS,nan,NaN,NaN,NaN,NaN,Ajeng,NaN,NaN,NaN,Passed,NaN,NaN,NaN
8,[PIB-41] Force Close,Mega Menu,Mega Menu,pada saat melakukan paste di field 'search' te...,Seharusnya tidak force closed,Android,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Passed,NaN,NaN,NaN
9,[PIB-41] Force Close,Mega Menu,Mega Menu,NaN,Seharusnya tidak force closed,iOS,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Failed,NaN,NaN,NaN


In [120]:
df.Features[0]

nan

In [146]:
version_chosen = version[0]
version_chosen

'PTR Ver APK 1.0.5 IPA 1.0.5'

In [147]:
df = df.drop(columns=[col for col in df.columns if (col.startswith('Status ') or col.startswith('Tanggal Eksekusi ') or col.startswith('Tanggal Passed ')) \
    and col not in ['Status ' + version_chosen, 'Tanggal Eksekusi ' + version_chosen, 'Tanggal Passed ' + version_chosen]])

In [148]:
df.head(3)

,Link JIRA,Features,Sub-features,Problem Description,Expected Condition,OS,OS Version,Tipe Device HP,Telko Provider HP,Rekening Sumber,Data yang digunakan,PO,FT,Link Report Test,Description Issue dan Evidence,Tanggal Eksekusi PTR Ver APK 1.0.5 IPA 1.0.5,Tanggal Passed PTR Ver APK 1.0.5 IPA 1.0.5,Komentar
0,[PIB-42] Donwload dan Share Resi Transaksi,Share Resi,Share Resi,Nama Nasabah : xxxxxxxxxxxxxxxxANI\nNomor Reke...,Nasabah bisa download dan share resi transaksi,Android,nan,NaN,NaN,NaN,NaN,Ori,NaN,NaN,NaN,NaN,NaN,NaN
1,[PIB-42] Donwload dan Share Resi Transaksi,Share Resi,Share Resi,NaN,Nasabah bisa download dan share resi transaksi,iOS,nan,NaN,NaN,NaN,NaN,Ori,NaN,NaN,NaN,NaN,NaN,NaN
2,[PIB-106] Gagal Permintaan Kartu Debit,Manajemen Kartu,Manajemen Kartu,"Mohon Bantuannya, terdapat kegagalan request k...",Nasabah bisa request kartu debit di iOS dan An...,Android,nan,NaN,NaN,NaN,NaN,"Rifatun, Indah",NaN,NaN,NaN,NaN,NaN,NaN


## Bikin Heatmap

In [109]:
df_heat = pd.read_excel(file_data, '-', header=None)
df_heat

,0,1,2,3,4,5,6
0,NaN,Execution Android,Passed Android,Failed Android,N/A Android,In Progress Android,Pass with Noted Android
1,Impacted Fitur - PO (1.03 PTR),1,0.941176,0.058824,0,0,0
2,Fitur Baru - PO (1.0.5 PTR),1,0.904762,0.047619,0,0.047619,0
3,Existing Features - PO (1.0.5 PTR),1,0.928571,0.026786,0.008929,0.026786,0.017857
4,Hotfix - PO (1.0.5 PTR),1,0.928571,0,0,0,0.071429
5,Testing PTR - Tester (1.0.5 PTR),1,0.921875,0.015625,0.03125,0.015625,0.015625
6,Testing PTR Backoffice - Tester (1.0.5 PTR),1,0.903226,0.064516,0,0,0.032258
7,Tiket Ready PTR - Tester (1.0.5 PTR),1,0.555556,0.111111,0,0.222222,0.111111
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df1 = df_heat.loc[0:7]
df2 = df_heat.loc[10:17]
df3 = df_heat.loc[19:21]

In [111]:
df1.columns = df1.loc[0]
df2.columns = df2.loc[10]
df3.columns = df3.loc[19]

df1.drop(df1.index[0], inplace=True)
df1.rename(columns={df1.columns[0]: 'Sheet name'}, inplace=True)

df2.drop(df2.index[0], inplace=True)
df2.rename(columns={df2.columns[0]: 'Sheet name'}, inplace=True)

df3.drop(df3.index[0], inplace=True)
df3.rename(columns={df3.columns[0]: 'Sheet name'}, inplace=True)

C:\Users\nahar\AppData\Local\Temp\ipykernel_6632\1922139577.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nahar\AppData\Local\Temp\ipykernel_6632\1922139577.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nahar\AppData\Local\Temp\ipykernel_6632\1922139577.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nahar\AppData\Local\Temp\ipykernel_6632\1922139577.py:9: SettingWithCopyWarning:


A value is 

In [112]:
df1.iloc[:, 1:] *= 100
df2.iloc[:, 1:] *= 100
df3.iloc[:, 1:] *= 100

C:\Users\nahar\AppData\Local\Temp\ipykernel_6632\2497172325.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nahar\AppData\Local\Temp\ipykernel_6632\2497172325.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nahar\AppData\Local\Temp\ipykernel_6632\2497172325.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [97]:
df1

,Sheet name,Execution Android,Passed Android,Failed Android,N/A Android,In Progress Android,Pass with Noted Android
1,Impacted Fitur - PO (1.03 PTR),100,94.117647,5.882353,0,0,0
2,Fitur Baru - PO (1.0.5 PTR),100,90.47619,4.761905,0,4.761905,0
3,Existing Features - PO (1.0.5 PTR),100,92.857143,2.678571,0.892857,2.678571,1.785714
4,Hotfix - PO (1.0.5 PTR),100,92.857143,0,0,0,7.142857
5,Testing PTR - Tester (1.0.5 PTR),100,92.1875,1.5625,3.125,1.5625,1.5625
6,Testing PTR Backoffice - Tester (1.0.5 PTR),100,90.322581,6.451613,0,0,3.225806
7,Tiket Ready PTR - Tester (1.0.5 PTR),100,55.555556,11.111111,0,22.222222,11.111111


In [ ]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go


df_heat = pd.read_excel(file_data, '-', header=None)

df1 = df_heat.loc[0:7]
df2 = df_heat.loc[10:17]
df3 = df_heat.loc[19:21]

df1.columns = df1.loc[0]
df2.columns = df2.loc[10]
df3.columns = df3.loc[19]

df1.drop(df1.index[0], inplace=True)
df1.rename(columns={df1.columns[0]: 'Sheet name'}, inplace=True)

df2.drop(df2.index[0], inplace=True)
df2.rename(columns={df2.columns[0]: 'Sheet name'}, inplace=True)

df3.drop(df3.index[0], inplace=True)
df3.rename(columns={df3.columns[0]: 'Sheet name'}, inplace=True)

df1.iloc[:, 1:] *= 100
df2.iloc[:, 1:] *= 100
df3.iloc[:, 1:] *= 100

# Melt dataframes for heatmap preparation
df1_melted = df1.melt(id_vars="Sheet name", var_name="Metric", value_name="Value")
df2_melted = df2.melt(id_vars="Sheet name", var_name="Metric", value_name="Value")
df3_melted = df3.melt(id_vars="Sheet name", var_name="Metric", value_name="Value")

# Pivot the melted dataframes
heatmap_data1 = df1_melted.pivot(index="Sheet name", columns="Metric", values="Value")
heatmap_data2 = df2_melted.pivot(index="Sheet name", columns="Metric", values="Value")
heatmap_data3 = df3_melted.pivot(index="Sheet name", columns="Metric", values="Value")

# Create subplots
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Android Metrics", "iOS Metrics", "Backoffice Metrics"),
    vertical_spacing=0.15
)

# Define a custom colorscale
custom_colorscale = [
    [0, "#f2f2f2"],  # Light gray
    [0.25, "#add8e6"],  # Light blue
    [0.5, "#87ceeb"],  # Sky blue
    [0.75, "#4682b4"],  # Steel blue
    [1, "#000080"],  # Navy
]

# Add heatmaps with text annotations to subplots
fig.add_trace(
    go.Heatmap(
        z=heatmap_data1.values,
        x=heatmap_data1.columns,
        y=heatmap_data1.index,
        colorscale=custom_colorscale,
        showscale=True if False else True,  # Single color legend
        colorbar=dict(title="Metrics", thickness=15, len=0.3, x=1.02),
        text=heatmap_data1.values,  # Add text data
        texttemplate="%{text:.2f}",  # Format text (2 decimal places)
        textfont=dict(color="white"),  # Set text color
    ),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=heatmap_data2.values,
        x=heatmap_data2.columns,
        y=heatmap_data2.index,
        colorscale=custom_colorscale,
        showscale=False,  # No separate legend for this heatmap
        text=heatmap_data2.values,  # Add text data
        texttemplate="%{text:.2f}",  # Format text (2 decimal places)
        textfont=dict(color="white"),  # Set text color
    ),
    row=2, col=1
)

fig.add_trace(
    go.Heatmap(
        z=heatmap_data3.values,
        x=heatmap_data3.columns,
        y=heatmap_data3.index,
        colorscale=custom_colorscale,
        showscale=False,  # No separate legend for this heatmap
        text=heatmap_data3.values,  # Add text data
        texttemplate="%{text:.2f}",  # Format text (2 decimal places)
        textfont=dict(color="white"),  # Set text color
    ),
    row=3, col=1
)

# Update layout
fig.update_layout(
    title_text="Heatmap Comparison: Android, iOS, and Backoffice Metrics",
    height=900,
    width=800,
    template="plotly_dark",  # Apply dark theme
    font=dict(size=12, color="white"),
    title_font=dict(size=18, color="white"),
    plot_bgcolor="#111111",  # Dark background
    paper_bgcolor="#111111",  # Dark background
)


fig.show()


In [108]:

df3_melted = df3.melt(id_vars="Sheet name", var_name="Metric", value_name="Value")
heatmap_data3 = df3_melted.pivot(index="Sheet name", columns="Metric", values="Value")

# Update Backoffice heatmap in the subplot
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Android Metrics", "iOS Metrics", "Backoffice Metrics"),
    horizontal_spacing=0.2
)

# Add Android heatmap
fig.add_trace(
    go.Heatmap(
        z=heatmap_data1.values,
        x=heatmap_data1.columns,
        y=heatmap_data1.index,
        colorscale="Viridis",
        colorbar=dict(title="Android", x=0.29)
    ),
    row=1, col=1
)

# Add iOS heatmap
fig.add_trace(
    go.Heatmap(
        z=heatmap_data2.values,
        x=heatmap_data2.columns,
        y=heatmap_data2.index,
        colorscale="Cividis",
        colorbar=dict(title="iOS", x=0.65)
    ),
    row=1, col=2
)

# Add Backoffice heatmap (with 2 sheet names)
fig.add_trace(
    go.Heatmap(
        z=heatmap_data3.values,
        x=heatmap_data3.columns,
        y=heatmap_data3.index,
        colorscale="Plasma",
        colorbar=dict(title="Backoffice", x=1.02)
    ),
    row=1, col=3
)

# Update layout
fig.update_layout(
    title_text="Heatmap Comparison: Android, iOS, and Backoffice Metrics (Backoffice: 2 Sheets)",
    height=600,
    width=1200
)

fig.show()


## Test Sankey Diagram

In [150]:

# Determine the primary column to use: "Link JIRA" if it exists, otherwise "Features"
primary_column = "Link JIRA" if "Link JIRA" in excel_ptr.columns else "Features"

# Generate unique nodes from relevant columns
nodes = list(set(
    excel_ptr[primary_column].tolist() +
    excel_ptr['Sub-features'].tolist() +
    excel_ptr['OS'].tolist() +
    excel_ptr['OS Version'].tolist() +
    excel_ptr['Tipe Device HP'].tolist() +
    excel_ptr["Status " + version_chosen].tolist()
))

def wrap_long_name(name, width=50):
    return '<br>'.join(textwrap.wrap(str(name), width))

# Create short labels for Sankey plot without modifying the original dataframe
node_indices = {
    node: (str(node)[:30] + "...") if isinstance(node, str) and len(str(node)) > 30 else str(node)
    for node in nodes
}

short_nodes = [node_indices[node] for node in nodes]
long_nodes = [wrap_long_name(node) for node in nodes]

# Create flows (source → target)
sources = []
targets = []
values = []

# Generate sources and targets for the Sankey diagram
for _, row in excel_ptr.iterrows():
    primary_value = row[primary_column]
    sub_feature = row["Sub-features"]
    status = row["Status " + version_chosen]
    os_type = row["OS"]

    # Primary Column -> Status -> OS if Status is "Passed"
    if status == "Passed":
        sources.append(nodes.index(primary_value))  # Primary Column -> Status
        targets.append(nodes.index(status))
        values.append(1)

        sources.append(nodes.index(status))  # Status -> OS
        targets.append(nodes.index(os_type))
        values.append(1)

    # Primary Column -> Sub-feature -> Status -> OS for other statuses
    elif status in ["Failed", "N/A", "In Progress", "Not Started"]:
        sources.append(nodes.index(primary_value))  # Primary Column -> Sub-feature
        targets.append(nodes.index(sub_feature))
        values.append(1)

        sources.append(nodes.index(sub_feature))  # Sub-feature -> Status
        targets.append(nodes.index(status))
        values.append(1)

        sources.append(nodes.index(status))  # Status -> OS
        targets.append(nodes.index(os_type))
        values.append(1)

# Calculate incoming and outgoing flows
incoming_flows = {node: 0 for node in nodes}
outgoing_flows = {node: 0 for node in nodes}

for source, target in zip(sources, targets):
    outgoing_flows[nodes[source]] += 1
    incoming_flows[nodes[target]] += 1

# Prepare customdata with both incoming and outgoing flows
customdata = [
    f"{long_name} <br>Incoming: {incoming_flows[node]} <br>Outgoing: {outgoing_flows[node]}"
    for node, long_name in zip(nodes, long_nodes)
]

# Assign default value of 1 for each connection
values = [1] * len(sources)

# Define colors for nodes and links
color_palette = px.colors.qualitative.Light24
num_colors = len(color_palette)
opacity = 1
node_colors = {}

# Assign colors to features
for i, primary_value in enumerate(excel_ptr[primary_column].unique()):
    hex_color = color_palette[i % num_colors]
    rgba_color = to_rgba(hex_color, alpha=opacity)
    node_colors[primary_value] = f"rgba({int(rgba_color[0]*255)}, {int(rgba_color[1]*255)}, {int(rgba_color[2]*255)}, {rgba_color[3]})"

# Propagate feature colors to sub-features
for primary_value in excel_ptr[primary_column].unique():
    feature_color = node_colors[primary_value]
    for sub_feature in excel_ptr[excel_ptr[primary_column] == primary_value]["Sub-features"].unique():
        node_colors[sub_feature] = "rgba(200, 200, 200, 0.8)"

# Overwrite colors for specific statuses
node_colors["Passed"] = "rgba(144, 238, 144, 0.8)"
node_colors["Failed"] = "rgba(205, 92, 92, 0.8)"
node_colors["Android"] = "#71BC68"
node_colors["iOS"] = "rgba(70, 130, 180, 0.8)"
default_color = "rgba(200, 200, 200, 0.8)"
node_color_list = [node_colors.get(node, default_color) for node in nodes]

# Assign link colors based on source node color
link_colors = []
for source, target in zip(sources, targets):
    source_color = node_colors.get(nodes[source], "rgba(192, 192, 192, 0.3)")
    rgba_values = source_color.strip("rgba()").split(",")
    if len(rgba_values) == 4:
        r, g, b, _ = map(float, rgba_values[:4])
        link_colors.append(f"rgba({int(r)}, {int(g)}, {int(b)}, 0.3)")
    else:
        link_colors.append("rgba(192, 192, 192, 0.3)")

# Plot Sankey Diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=short_nodes,
        color=node_color_list,
        customdata=customdata,
        hovertemplate="%{customdata}<extra></extra>"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors
    )
)])

# Update layout and show
fig.update_layout(
    font_size=12,
    height=500,
    font=dict(size=14, color='white'),
    plot_bgcolor='#1E1E1E',
    paper_bgcolor='#1E1E1E',
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()

KeyError: 'Status PTR Ver APK 1.0.5 IPA 1.0.5'

In [89]:
df_android = df[df['OS'] == 'Android']

In [90]:
import numpy as np
df_android['Status '+ version_chosen] = df_android['Status ' + version_chosen].replace(np.nan, 'Not Started')

C:\Users\nahar\AppData\Local\Temp\ipykernel_25268\1312768736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android['Status '+ version_chosen] = df_android['Status ' + version_chosen].replace(np.nan, 'Not Started')


In [91]:
df_android['Status ' + version_chosen].value_counts().apply(lambda x:x/len(df_android)*100)

Status PTR Ver APK 1.0.5 IPA 1.0.5
Passed         90.476190
In Progress     4.761905
Failed          4.761905
Name: count, dtype: float64

In [108]:
def progress_status(df, version):
    df_android = df[df['OS'] == 'Android'].copy()
    df_ios = df[df['OS'] == 'iOS'].copy()

    def process_data(data, version):
        data['Status ' + version] = data['Status ' + version].replace(np.nan, 'N/A')
        return data['Status ' + version].value_counts(normalize=True) * 100

    android_result = process_data(df_android, version)
    ios_result = process_data(df_ios, version)
    
    df_plot = pd.concat([android_result.rename("Android"), ios_result.rename('iOS')], axis=1).reset_index()
    df_plot.rename(columns={'Status '+version : 'Status'}, inplace=True)
    
    df_plot = df_plot.melt(id_vars='Status', var_name='Platform', value_name='Percentage')
    df_plot['Percentage'] = df_plot['Percentage'].apply(lambda row: round(row, 2))

    return df_plot

def progress_plot(df_plot, version):
    progress_bar = px.bar(
        df_plot,
        x="Status",
        y="Percentage",
        color="Platform",
        barmode="group",
        title=f"Status Distribution by Platform (Version {version})",
        text="Percentage",
        color_discrete_map={"Android": "#FFA500", "iOS": "#1E90FF"}  # Custom colors
    )

    # Customize the traces for better readability
    progress_bar.update_traces(
        texttemplate='%{text:.2f}%',
        textposition='outside',
        marker=dict(line=dict(width=1.5, color="black"))  # Add a border to bars
    )

    # Update layout for a cleaner and professional look
    progress_bar.update_layout(
        title=dict(
            text=f"<b>Status Distribution by Platform (Version {version})</b>",
            font=dict(size=20, family="Arial, sans-serif"),
            x=0.5  # Center the title
        ),
        xaxis=dict(
            title="<b>Status</b>",
            tickangle=-45,  # Tilt x-axis labels for readability
            tickfont=dict(size=12, family="Arial, sans-serif"),
        ),
        yaxis=dict(
            title="<b>Percentage (%)</b>",
            tickfont=dict(size=12, family="Arial, sans-serif"),
        ),
        legend=dict(
            title="<b>Platform</b>",
            font=dict(size=12, family="Arial, sans-serif"),
            bgcolor="#1E1E1E",  # Light gray background for legend
            bordercolor="black",
            borderwidth=1,
        ),
        margin=dict(l=50, r=50, t=80, b=50),  # Adjust margins for spacing
        plot_bgcolor="#1E1E1E",  # Light gray background
        paper_bgcolor="#1E1E1E",  # Light gray background
        width=1000,
        height=600
    )
    
    return progress_bar

In [109]:
df_plot = progress_status(df=df, version=version_chosen)
df_plot

,Status,Platform,Percentage
0,Passed,Android,38.89
1,N/A,Android,22.22
2,Failed,Android,16.67
3,Not Started,Android,16.67
4,In Progress,Android,5.56
5,Passed,iOS,52.63
6,N/A,iOS,5.26
7,Failed,iOS,15.79
8,Not Started,iOS,26.32
9,In Progress,iOS,NaN


In [114]:
df_plot[df_plot['Platform'] == 'Android']['Percentage'].tolist()

[38.89, 22.22, 16.67, 16.67, 5.56]

In [120]:
z

Status,Failed,In Progress,N/A,Not Started,Passed
Platform,,,,,
Android,16.67,5.56,22.22,16.67,38.89
iOS,15.79,0.00,5.26,26.32,52.63


In [127]:
df_plot[df_plot['Platform'] == 'Android']

,Status,Platform,Percentage
0,Passed,Android,38.89
1,N/A,Android,22.22
2,Failed,Android,16.67
3,Not Started,Android,16.67
4,In Progress,Android,5.56


In [134]:
android_data

,Status,Platform,Percentage
0,Passed,Android,38.89
1,N/A,Android,22.22
2,Failed,Android,16.67
3,Not Started,Android,16.67
4,In Progress,Android,5.56


In [135]:
z_df

Status,Passed,Failed,Not Started,N/A,In Progress,Blocked,Pass with Noted
Sheet Name,,,,,,,
Prototype - Tiket Prod Issue PO,38.89,0,0,0,0,0,0


In [147]:
pd.Categorical(android_data['Status'], categories=status_fixed, ordered=True)

['Passed', 'N/A', 'Failed', 'Not Started', 'In Progress']
Categories (7, object): ['Passed' < 'Failed' < 'Not Started' < 'N/A' < 'In Progress' < 'Blocked' < 'Pass with Noted']

In [151]:
android_data = df_plot[df_plot['Platform'] == 'Android'].copy()  # Ensure this is a proper copy

# Apply pd.Categorical to permanently update the column
android_data['Status'] = pd.Categorical(android_data['Status'], categories=status_fixed, ordered=True)

# Verify the change
print(android_data['Status'])


0         Passed
1            N/A
2         Failed
3    Not Started
4    In Progress
Name: Status, dtype: category
Categories (7, object): ['Passed' < 'Failed' < 'Not Started' < 'N/A' < 'In Progress' < 'Blocked' < 'Pass with Noted']


In [154]:
print(android_data['Status'].dtype)


category


In [155]:
import pandas as pd
import plotly.graph_objects as go

# Fixed list of statuses
status_fixed = ['Passed', 'Failed', 'Not Started', 'N/A', 'In Progress', 'Blocked', 'Pass with Noted']

# Define the external `sheetname` variable for y-axis
sheetname = ['Prototype - Tiket Prod Issue PO']  # Add more sheet names as needed

android_data = df_plot[df_plot['Platform'] == 'Android'].copy()  # Ensure this is a proper copy

# Apply pd.Categorical to permanently update the column
android_data['Status'] = pd.Categorical(android_data['Status'], categories=status_fixed, ordered=True)


# Prepare the heatmap data
z_df = android_data.pivot_table(
    index=pd.Series(sheetname, name='Sheet Name'),
    columns='Status',
    values='Percentage',
    aggfunc='mean',
    observed=False  # Explicitly specify the behavior for categorical grouping
).reindex(columns=status_fixed, fill_value=0)      # Reindex to include all statuses, fill missing values with 0

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_df.values,        # Heatmap values
    x=z_df.columns,       # Fixed statuses for x-axis
    y=z_df.index,         # Sheet names for y-axis
    colorscale='Viridis', # Color scale
    showscale=False,      # Hide color scale to make it table-like
    texttemplate="%{z:.2f}%",  # Display values with 2 decimal points
    textfont={"size": 14},     # Adjust font size for readability
))

# Update layout for table styling
fig.update_layout(
    title=dict(text='Android Platform Status Table', font=dict(size=20)),
    xaxis=dict(
        title='Status',
        tickfont=dict(size=14),
        showgrid=True,       # Show grid lines to mimic table borders
        zeroline=False,
    ),
    yaxis=dict(
        title='Sheet Name',
        tickfont=dict(size=14),
        showgrid=True,
        zeroline=False,
    ),
    margin=dict(l=50, r=50, t=50, b=50),
    height=400 + 20 * len(sheetname),  # Dynamically adjust height for multiple sheetnames
    plot_bgcolor="white",              # White background for table look
    paper_bgcolor="white",
)

# Add gridline style to make the plot look like a table
fig.update_traces(
    hoverinfo="text",   # Display only text on hover
    xgap=5,             # Space between x-axis cells
    ygap=5,             # Space between y-axis cells
    showscale=False     # Hide the color scale
)

fig.show()


In [20]:
progress_plot(df_plot=df_plot, version=version_chosen)

In [48]:
def truncate_and_wrap(text, max_words=5):
    words = text.split()
    if len(words) > max_words:
        first_line = " ".join(words[:max_words])
        second_line = " ".join(words[max_words:max_words*2])
        truncated_text = f"{first_line}\n{second_line}"
    else:
        truncated_text = " ".join(words)
    return truncated_text

def normalize_text(text):
    if isinstance(text, str):
        return text.strip().lower()
    else:
        return str(text).lower()  # Convert to string if it's not already, then normalize

In [53]:
import textwrap

def wrap_text(text, width=50):
    return '\n'.join(textwrap.wrap(text, width))

In [ ]:
node_indices[]

KeyError: 2

In [133]:
nodes = list(set(
    df['Features'].tolist() +
    df['Sub-features'].tolist() +
    df['OS'].tolist() +
    df['OS Version'].tolist() +
    df['Tipe Device HP'].tolist() +
    df["Status "+ version_chosen].tolist()
))

def wrap_long_name(name, width=50):
    return '<br>'.join(textwrap.wrap(str(name), width))

# Create short labels for Sankey plot without modifying the original df
node_indices = {
    node: (str(node)[:20] + "...") if isinstance(node, str) and len(str(node)) > 20 else str(node)
    for node in nodes
}

# Create a list of shortened node labels for the Sankey plot
short_nodes = [node_indices[node] for node in nodes]

long_nodes = [wrap_long_name(node) for node in nodes]

# Create flows (source → target)
sources = []
targets = []
values = []

# Generate sources and targets for the Sankey diagram
for _, row in df.iterrows():
    feature = row["Features"]
    sub_feature = row["Sub-features"]
    status = row["Status "+ version_chosen]
    os_type = row["OS"]

    # Feature -> Status -> OS if Status is "Passed"
    if status == "Passed":
        sources.append(nodes.index(feature))  # Feature -> Status
        targets.append(nodes.index(status))
        values.append(1)

        sources.append(nodes.index(status))  # Status -> OS
        targets.append(nodes.index(os_type))
        values.append(1)

    # Feature -> Sub-feature -> Status -> OS if Status is "Failed"
    elif status == "Failed" or status == "N/A" or status == "In Progress" or status == "Not Started":
        sources.append(nodes.index(feature))  # Feature -> Sub-feature
        targets.append(nodes.index(sub_feature))
        values.append(1)

        sources.append(nodes.index(sub_feature))  # Sub-feature -> Status
        targets.append(nodes.index(status))
        values.append(1)

        sources.append(nodes.index(status))  # Status -> OS
        targets.append(nodes.index(os_type))
        values.append(1)
        
    # Calculate incoming and outgoing flows
    incoming_flows = {node: 0 for node in nodes}
    outgoing_flows = {node: 0 for node in nodes}

    for source, target in zip(sources, targets):
        outgoing_flows[nodes[source]] += 1  # Increase the outgoing flow count for the source node
        incoming_flows[nodes[target]] += 1  # Increase the incoming flow count for the target node

    # Prepare customdata with both incoming and outgoing flows
    customdata = [
        f"{long_name} <br>Incoming: {incoming_flows[node]} <br>Outgoing: {outgoing_flows[node]}"
        for node, long_name in zip(nodes, long_nodes)
    ]
        
    # Assign a default value of 1 for each connection
    values = [1] * len(sources)

    # Use a qualitative color scale for high contrast (e.g., Plotly's 'Dark24')
    color_palette = px.colors.qualitative.Light24 # Replace with another palette if needed
    num_colors = len(color_palette)

    opacity = 1  # Example opacity value (0.0 - 1.0)
    node_colors = {}
    for i, feature in enumerate(df["Features"].unique()):
        hex_color = color_palette[i % num_colors]  # Get the hex color
        rgba_color = to_rgba(hex_color, alpha=opacity)  # Convert to RGBA
        node_colors[feature] = f"rgba({int(rgba_color[0]*255)}, {int(rgba_color[1]*255)}, {int(rgba_color[2]*255)}, {rgba_color[3]})"

    # Propagate feature colors to sub-features
    for feature in df["Features"].unique():
        feature_color = node_colors[feature]
        for sub_feature in df[df["Features"] == feature]["Sub-features"].unique():
            node_colors[sub_feature] = feature_color

    # Overwrite colors for 'Passed' and 'Failed'
    node_colors["Passed"] = "rgba(144, 238, 144, 0.8)"  # Soft green
    node_colors["Failed"] = "rgba(205, 92, 92, 0.8)"    # Soft red brick

    node_colors["Android"] = "rgba(64, 224, 208, 0.8)"  # Green turquoise
    node_colors["iOS"] = "rgba(70, 130, 180, 0.8)"      # Steel blue

    # Set default color for unassigned nodes
    default_color = "rgba(200, 200, 200, 0.8)"
    node_color_list = [node_colors.get(node, default_color) for node in nodes]

    # Assign link colors based on source node color with transparency
    link_colors = []
    for source, target in zip(sources, targets):
        source_color = node_colors.get(nodes[source], "rgba(192, 192, 192, 0.3)")  # Default gray if missing
        rgba_values = source_color.strip("rgba()").split(",")
        if len(rgba_values) == 4:
            r, g, b, _ = map(float, rgba_values[:4])
            link_colors.append(f"rgba({int(r)}, {int(g)}, {int(b)}, 0.3)")  # Reduce opacity for the links
        else:
            link_colors.append("rgba(192, 192, 192, 0.3)")  # Default gray


# Plot Sankey Diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=short_nodes,  # Use short labels here
        color=node_color_list,  # Optionally set a default color
        customdata=customdata,
        hovertemplate="%{customdata}<extra></extra>"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors
    )
)])

# Update layout and show
fig.update_layout(
    font_size=12,
    width=1000,  # Increase width for a wider graph
    height=700,   # Increase height for a taller graph
    font=dict(size=14, color='white'),
    plot_bgcolor='#1E1E1E',
    paper_bgcolor='#1E1E1E',
    margin=dict(l=20, r=20, t=20, b=20)
)

fig.show()


In [134]:
link_colors

['rgba(253, 50, 22, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(0, 254, 53, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(0, 254, 53, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(106, 118, 252, 0.3)',
 'rgba(214, 38, 255, 0.3)',
 'rgba(205, 92, 92, 0.3)',
 'rgba(106, 118, 252, 0.3)',
 'rgba(214, 38, 255, 0.3)',
 'rgba(205, 92, 92, 0.3)',
 'rgba(254, 212, 196, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(254, 212, 196, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(254, 0, 206, 0.3)',
 'rgba(254, 0, 206, 0.3)',
 'rgba(205, 92, 92, 0.3)',
 'rgba(254, 0, 206, 0.3)',
 'rgba(254, 0, 206, 0.3)',
 'rgba(205, 92, 92, 0.3)',
 'rgba(13, 249, 255, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(13, 249, 255, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(246, 249, 38, 0.3)',
 'rgba(255, 150, 22, 0.3)',
 'rgba(192, 192, 192, 0.3)',
 'rgba(255, 150, 22, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(71, 155, 85, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(71, 155, 85, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(238, 166, 251

In [125]:
long_nodes[1]

'18.01'

In [117]:
df.Features[0]

'PS-495 : RELEASE A - IOS - Tidak bisa klik banner donasi - IIN'